In [1]:
import {Complex,creal,cstr,cadd,cmul,cinv,cabs,cdiv} from './comp.js'
import {Polynomial,pstr,pdeg,peval,pdiff,pGCD} from './poly.js'

In [2]:
var p = new Polynomial([creal(6),creal(5),creal(1)])

In [3]:
var plin = pdiff(p)

In [4]:
pdeg(pGCD(p,plin))

0


In [49]:
function abStep(poly,guess) {
    if(pdeg(poly)!=guess.length) {return 0}
    var dp=pdiff(poly),dpz,pz,z,s;
    var update=[],i,j,ps,zs,os;
    for(i=0;i<guess.length;i++) {
        z = guess[i];
        pz = peval(poly,z);
        if(cabs(pz)<1e-10) {
            update.push(z);
        } else {
            ps = creal(0);
            for(j=0;j<guess.length;j++) {
            if(i!=j) {
                s = guess[j];
                zs = cadd(z,cmul(creal(-1),s));
                if(cabs(zs)<1e-6) {return 0}
                ps = cadd(ps,cinv(zs));
            }
        }
        dpz = peval(dp,z);
        os = cadd(cdiv(dpz,pz),cmul(creal(-1),ps));            
        update.push(cadd(z,cmul(creal(-1),cinv(os))));
        }     
    }
    return update
}

function abRecur(poly,guess) {
    var update,i,change=0;
    update = abStep(poly,guess);
    if(update==0) {console.log("Error"); return}
    for(i=0;i<guess.length;i++) {
        change += cabs(cadd(update[i],cmul(creal(-1),guess[i])))
    }
    if (change<1e-10) {return update}
    return abRecur(poly,update)
}

In [50]:
abStep(p,[new Complex([2,1]),new Complex([4,-5])])

[ Complex { re: -0.06859382655561008, im: 1.2283194512493876 },
  Complex { re: -2.995121951219513, im: -5.022764227642275 } ]


In [51]:
var twop = new Polynomial([creal(-1.5),creal(-0.25),creal(1)])

In [52]:
abRecur(twop,[new Complex([2,1]),new Complex([4,-5])])

[ Complex { re: 1.356107225224484, im: 8.517756541325666e-15 },
  Complex { re: -1.1061072252244297, im: -6.484267604193322e-14 } ]


In [57]:
var threep = new Polynomial([creal(-1),creal(-0.75),creal(-0.25),creal(1)])

In [58]:
abRecur(threep,[new Complex([2,1]), new Complex([4,-5]), new Complex([0.25,-0.75])])

[ Complex { re: 1.3519136862464185, im: -1.1871094207397087e-13 },
  Complex { re: -0.5509568431238944, im: 0.6604080182084614 },
  Complex { re: -0.5509568431231111, im: -0.6604080182084816 } ]


In [59]:
var multip = new Polynomial([creal(1),creal(2),creal(1)])

In [60]:
abRecur(multip,[new Complex([2,1]),new Complex([4,-5])])

[ Complex { re: -0.9999968638743151, im: 0.0000031361251247801898 },
  Complex { re: -1.000003136125326, im: -0.0000031361333518193908 } ]
